In [12]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import nltk
import ast

# Download NLTK data (if not already downloaded)
#nltk.download('punkt')

# Read the CSV files
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Merge movies and credits dataframes
movies = movies.merge(credits, on='title')

# Select relevant columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Drop rows with missing values
movies.dropna(inplace=True)

# Convert genres, keywords, cast, and crew columns to lists of names
def convert(obj):
    names = []
    for item in ast.literal_eval(obj):
        names.append(item['name'])
    return names

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

def convertkey(obj):
    names = []
    counter = 0
    for item in ast.literal_eval(obj):
        if counter != 3:
            names.append(item['name'])
            counter += 1
        else:
            break
    return names

movies['cast'] = movies['cast'].apply(convertkey)

def fetch_director(obj):
    names = []
    for item in ast.literal_eval(obj):
        if item['job'] == 'Director':
            names.append(item['name'])
            break
    return names

movies['crew'] = movies['crew'].apply(fetch_director)

# Preprocess the overview column
stemmer = PorterStemmer()

def preprocess_text(text):
    stemmed_words = [stemmer.stem(word) for word in nltk.word_tokenize(text.lower())]
    return " ".join(stemmed_words)

movies['overview'] = movies['overview'].apply(preprocess_text)

# Combine relevant columns into a new 'tags' column
movies['tags'] = movies['overview'] + ' ' + movies['genres'].apply(lambda x: ' '.join(x)) + ' ' + \
                  movies['keywords'].apply(lambda x: ' '.join(x)) + ' ' + \
                  movies['cast'].apply(lambda x: ' '.join(x)) + ' ' + movies['crew'].apply(lambda x: ' '.join(x))

# Initialize the CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

# Create the feature vectors
vectors = cv.fit_transform(movies['tags']).toarray()

# Compute cosine similarity
similarity = cosine_similarity(vectors)

# Function to recommend similar movies
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_indices = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = [i[0] for i in movie_indices]
    return recommended_movies

# Test the recommendation function
recommendations = recommend('Avengers: Age of Ultron')
print(recommendations)

[16, 79, 68, 26, 31]


In [4]:
import pickle
pickle.dump(movies,open('movies.pkl','wb'))

In [6]:
movies['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [8]:
pickle.dump(movies.to_dict(),open('movies_dict.pkl','wb'))

In [9]:
pickle.dump(similarity,open('similarity.pkl','wb'))